# Combine results from multiple ERDDAPs

In this notebook we use data from one ERDDAP (OSMC animal-borne sensors) to make subsequent queries to other ERDDAPs supplying Argo and satellite data

In [ ]:
import datetime
import pandas as pd
from erddapy import ERDDAP
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature


def cartopy_map(lon, lat):
    central_lon = (lon.min() + lon.max())/2
    central_lat = (lat.min() + lat.max())/2
    fig = plt.figure(figsize=[10, 10])
    ax = fig.add_subplot(projection=ccrs.AzimuthalEquidistant(central_latitude=central_lat, central_longitude=central_lon))
    ax.add_feature(cfeature.LAND)
    ax.add_feature(cfeature.OCEAN)
    ax.gridlines(color='k', alpha=0.2)
    #ax.set_extent([lon.min()-0.5, lon.max()+0.5, lat.min()-0.5, lat.max()+1.5], crs=ccrs.PlateCarree())
    return ax

### 1. MEOPS animals

We can use ERDDAP's built in search function to find datasets that match a keyword. Let's find some animal borne sensor data

In [ ]:
e_osmc = ERDDAP(
    server="https://osmc.noaa.gov/erddap",
    protocol="tabledap"
)
url = e_osmc.get_search_url(search_for="animal", response="csv")

animals = pd.read_csv(url)
animals[["Dataset ID", "Title", "Summary"]]

In [ ]:
e_osmc.dataset_id = animals["Dataset ID"][0]
e_osmc.variables = ['species']
species = e_osmc.to_pandas()
species

In [ ]:
e_osmc.constraints = {'species=': 'Southern elephant seal', #'Southern elephant seal', 'Crabeater seal'
                 'longitude>=': -90,
                 'longitude<=': -60,
                 'latitude<=': -60
}
e_osmc.variables = ['latitude', 'longitude', 'time', 'species']
df = e_osmc.to_pandas()
df

In [ ]:
ax = cartopy_map(df['longitude (degrees_east)'], df['latitude (degrees_north)'])
ax.scatter(df['longitude (degrees_east)'], df['latitude (degrees_north)'], transform=ccrs.PlateCarree(), color='k', s=1)


### 2. Add argo data

We use a bounding box from the lon, lat and time of the seal data to look for co-located Argo floats on the ifremer ERDDAP

In [ ]:
e_ifremer = ERDDAP(
    server="https://erddap.ifremer.fr/erddap",
    protocol="tabledap"
)
e_ifremer.dataset_id = "ArgoFloats"

In [ ]:
e_ifremer.constraints = {'time>=': df["time (UTC)"].min(),
 'time<=': df["time (UTC)"].max(),
 'latitude>=':  df['latitude (degrees_north)'].min(),
 'latitude<=':  df['latitude (degrees_north)'].max(),
 'longitude>=': df['longitude (degrees_east)'].min(),
 'longitude<=': df['longitude (degrees_east)'].max(),}
e_ifremer.variables = ['latitude', 'longitude', 'time']

In [ ]:
df_argo = e_ifremer.to_pandas()
df_argo

In [ ]:
ax = cartopy_map(df['longitude (degrees_east)'], df['latitude (degrees_north)'])
ax.scatter(df['longitude (degrees_east)'], df['latitude (degrees_north)'], transform=ccrs.PlateCarree(), color='k', s=1, label="seals")
ax.scatter(df_argo['longitude (degrees_east)'], df_argo['latitude (degrees_north)'], transform=ccrs.PlateCarree(), color="r", s=3, label="Argo")
ax.legend()

### 3. Gridded SST

Finally, we get some matching JPL reanalysis SST data from the coastwatch ERDDAP 

In [ ]:
coastwatch_e = ERDDAP(
    server="https://coastwatch.pfeg.noaa.gov/erddap",
    protocol="griddap",
)
coastwatch_e.dataset_id='jplMURSST41'
coastwatch_e.griddap_initialize() 

In [ ]:
coastwatch_e.constraints = {'time>=': df["time (UTC)"].min(),
 'time<=': df["time (UTC)"].max(),
 'time_step': 365,
 'latitude>=':  df['latitude (degrees_north)'].min(),
 'latitude<=':  df['latitude (degrees_north)'].max(),
 'latitude_step': 10,
 'longitude>=': df['longitude (degrees_east)'].min(),
 'longitude<=': df['longitude (degrees_east)'].max(),
 'longitude_step': 10}

coastwatch_e.variables=['analysed_sst']

In [ ]:
ds = coastwatch_e.to_xarray()

In [ ]:
ax = cartopy_map(df['longitude (degrees_east)'], df['latitude (degrees_north)'])
ax.contourf(ds.longitude, ds.latitude, ds.analysed_sst[0,:,:], transform=ccrs.PlateCarree(), cmap="RdBu")
ax.scatter(df['longitude (degrees_east)'], df['latitude (degrees_north)'], transform=ccrs.PlateCarree(), color='k', s=1, label="seals")
ax.scatter(df_argo['longitude (degrees_east)'], df_argo['latitude (degrees_north)'], transform=ccrs.PlateCarree(), color="r", s=5, label="Argo")
ax.legend()

**Bonus**: change the species to 'Crabeater seal' and re-run the notebook

---------------------------
### References

MEOP data from https://meop.net

Argo data from ifremer https://erddap.ifremer.fr/erddap/tabledap/ArgoFloats.html

SST reanalysis from coastwatch https://coastwatch.pfeg.noaa.gov/erddap

More info on using ERDDAP's inbuilt search https://ioos.github.io/erddapy/01b-tabledap-output.html